# Initialisation

In [ ]:
from IPython.core.display import HTML
HTML('<style> .container{ width:90%; } </style>')

Installing standard libraries, which only needs to be done once:
- Scientific computing (numpy)
- Data analysis (pandas)
- Plotting (matplotlib, seaborn)
- Machine learning (scikit-learn)
- Gradient-boosted trees (xgboost. lightgbm)
- Deep learning (tensorflow, keras, keras_metrics)

Uncomment for first run!!

In [ ]:
'''import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install xgboost
!{sys.executable} -m pip install lightgbm
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install keras_metrics
!{sys.executable} -m pip install h5py'''

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Data Loading

Loading CVS file onto pandas dataframe, a tabular data structure.

Printing first few rows, the shape of data (rows, column), the data types, and some basic analysis of the numeric features

In [ ]:
col_names = [
    'age', 
    'workclass', 
    'fnlwgt', 
    'education', 
    'education-num', 
    'marital-status', 
    'occupation', 
    'relationship',
    'race', 
    'sex', 
    'capital-gain', 
    'capital-loss', 
    'hours-per-week', 
    'native-country', 
    'income',
]

df = pd.read_csv('data/census_income.csv', names=col_names, skipinitialspace=True, na_values=['?'])
df.head(10)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

Based on the data description available at https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names, 
feature 'fnlwgt' does not add any useful information, so it is best to remove.

Feature "education" is also redundant because "education-num" encodes the same information with numerical values.

In [ ]:
df = df.drop('fnlwgt', axis=1)
df = df.drop('education', axis=1)

# Exploratory Data Analysis

Check whether numeric features are correlated. Since they are not, all of them bring new relevant information

In [ ]:
plt.figure(figsize=(10,4))
sns.heatmap(df.corr(), annot=True, cmap='Blues', linecolor='white', linewidths=1)
plt.show()

Plot data to understand it, determine whether any features look strongly correlated with high or low income

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x = df['income'], hue = df['education-num'], palette = 'rainbow', edgecolor = [(0,0,0), (0,0,0)])
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
sns.countplot(y = df['income'], hue = df['sex'], palette = 'Set1', edgecolor = [(0,0,0), (0,0,0)])
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
sns.countplot(y = df['income'], hue = df['marital-status'], palette = 'Set1', edgecolor = [(0,0,0), (0,0,0)])
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
sns.countplot(y = df['income'], hue = df['relationship'], palette = 'Set1', edgecolor = [(0,0,0), (0,0,0)])
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(y = df['income'], hue = df['occupation'], palette = 'tab20', edgecolor = [(0,0,0), (0,0,0)])
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
sns.countplot(y = df['income'], hue = df['race'], palette = 'Set1', edgecolor = [(0,0,0), (0,0,0)])
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
sns.countplot(y = df['income'], hue = df['workclass'], palette = 'Set1', edgecolor = [(0,0,0), (0,0,0)])
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxenplot(x='income', y='age', data=df, hue='sex', palette = 'prism')
plt.show()

# Dealing with NULLs

In [ ]:
df.info()

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
df['native-country'].value_counts().head(10)

In [ ]:
df['native-country'].fillna('United-States', inplace=True)
df[df['native-country'].isnull()]

In [ ]:
df['workclass'].value_counts()

In [ ]:
df['workclass'].fillna('Private', inplace=True)
df[df['workclass'].isnull()]

In [ ]:
df['occupation'].value_counts()

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
df.dropna(inplace=True)
df[pd.isnull(df).any(axis=1)]

# Transforming Categorical Features

In [ ]:
tdf = pd.get_dummies(df, columns=[
    'workclass', 
    'marital-status', 
    'occupation', 
    'relationship', 
    'race', 
    'sex',
    'native-country',
], drop_first=True)

In [ ]:
tdf.info()

# Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

Split the dataframe into features (X) and labels(y)

In [ ]:
X = tdf.drop('income', axis=1)
y = tdf['income']

Put aside 20% of features to test, train with remaining 80%

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Feature Selection

Now our problem is we have 80 variables to predict a single one. Many of these variables will have absolutely no impact on the income of our citizens, but will add a lot of useless noise to the regression, and may even make it not converge.

We use a little trick called SelectKBest that will give us the k=30 best features based on a standard statistical score. In this case, we use Pearson's Chi Squared test.

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

In [ ]:
feature_select = SelectKBest(chi2, k=30)
feature_select.fit(X_train, y_train)

List the surviving features, along with their scores

In [ ]:
uni_features = list(zip(feature_select.scores_, X_train.columns))
sorted(uni_features, reverse=True)[0:30]

# Utility Functions

In [ ]:
import warnings
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, classification_report, roc_auc_score, roc_curve
from sklearn.externals import joblib
from sklearn.exceptions import DataConversionWarning

warnings.filterwarnings(action='ignore', category=DataConversionWarning)
joblib.dump(list(X.columns), 'models/columns.pkl')

In [ ]:
def plot_metrics(classifier):
    prediction = classifier.predict(X_test)
    print('Accuracy is: %0.3f' % accuracy_score(y_test, prediction))
    print('F1 Score is: %0.3f' % f1_score(y_test, prediction, pos_label=">50K"))
    print(classification_report(y_test, prediction))
    sns.heatmap(confusion_matrix(y_test, prediction), annot=True, fmt="d")
    plt.show()

In [ ]:
def plot_roc_curve(classifier, classifier_name):
    try:
        prob_pos = classifier.predict_proba(X_test)[:,1]
    except:
        prob_pos = classifier.decision_function(X_test)
        prob_pos = (prob_pos - prob_pos.min()) / (prob_pos.max() - prob_pos.min())
        print('Probabilities estimated from decision function!')
    fpr, tpr, thresholds = roc_curve(y_test, prob_pos, pos_label=">50K")
    plt.figure()
    roc_score = roc_auc_score(y_test, classifier.predict(X_test)==">50K")
    plt.plot(fpr, tpr, label='%s (area = %0.3f)' % (classifier_name, roc_score))
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([-0.05, 1.05])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate = 1 - Specificity')
    plt.ylabel('True Positive Rate = Recall')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.savefig('img/ROC_' + classifier_name)
    plt.show()

In [ ]:
def plot_cap_curve(classifier, classifier_name):
    from scipy import integrate
    try:
        prob_pos = classifier.predict_proba(X_test)[:,1]
    except:
        prob_pos = classifier.decision_function(X_test)
        prob_pos = (prob_pos - prob_pos.min()) / (prob_pos.max() - prob_pos.min())
        print('Probabilities estimated from decision function!')
    y_values = y_test==">50K"
    num_pos_obs = np.sum(y_values)
    num_count = len(y_values)
    rate_pos_obs = float(num_pos_obs) / float(num_count)
    ideal = pd.DataFrame({'x':[0,rate_pos_obs,1],'y':[0,1,1]})
    xx = np.arange(num_count) / float(num_count - 1)
    
    y_cap = np.c_[y_values,prob_pos]
    y_cap_df_s = pd.DataFrame(data=y_cap)
    y_cap_df_s = y_cap_df_s.sort_values([1], ascending=False).reset_index(level = y_cap_df_s.index.names, drop=True)
    
    #print(y_cap_df_s.head(20))
    
    yy = np.cumsum(y_cap_df_s[0]) / float(num_pos_obs)
    yy = np.append([0], yy[0:num_count-1]) #add the first curve point (0,0) : for xx=0 we have yy=0
    
    percent = 0.5
    row_index = int(np.trunc(num_count * percent))
    
    val_y1 = yy[row_index]
    val_y2 = yy[row_index+1]
    if val_y1 == val_y2:
        val = val_y1*1.0
    else:
        val_x1 = xx[row_index]
        val_x2 = xx[row_index+1]
        val = val_y1 + ((val_x2 - percent)/(val_x2 - val_x1))*(val_y2 - val_y1)
    
    sigma_ideal = 1 * xx[num_pos_obs - 1 ] / 2 + (xx[num_count - 1] - xx[num_pos_obs]) * 1
    sigma_model = integrate.simps(yy,xx)
    sigma_random = integrate.simps(xx,xx)
    
    ar_value = (sigma_model - sigma_random) / (sigma_ideal - sigma_random)
    
    fig, ax = plt.subplots(nrows = 1, ncols = 1)
    ax.plot(ideal['x'],ideal['y'], color='dimgrey', label='Perfect Model')
    ax.plot(xx,yy, color='red', label="%s (AR = %0.3f)" % (classifier_name, ar_value))
    ax.plot(xx,xx, color='blue', label='Random Model')
    ax.plot([percent, percent], [0.0, val], color='green', linestyle='--', linewidth=1)
    ax.plot([0, percent], [val, val], color='green', linestyle='--', linewidth=1, label='%0.3f%% of positive obs at %0.1f%%' % (val*100, percent*100))
    
    plt.xlim(-0.05, 1.05)
    plt.ylim(0, 1.05)
    plt.title("Cumulative accuracy profile")
    plt.xlabel('% of the data')
    plt.ylabel('% of positive obs')
    plt.legend()
    plt.savefig('img/CAP_' + classifier_name)
    plt.show()

In [ ]:
def classify_and_plot(classifier):
    classifier_name = type(classifier).__name__
    if classifier_name == 'Pipeline':
        classifier_name = classifier.steps[-1][0]
    classifier.fit(X_train, y_train)
    plot_metrics(classifier)
    plot_roc_curve(classifier, classifier_name)
    plot_cap_curve(classifier, classifier_name)
    joblib.dump(classifier, 'models/' + classifier_name + '.pkl')
    return classifier

# Classic Data Science Classifiers

## K-nearest Neighbours Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
pipeline = Pipeline([
    ('select', feature_select),
    ('scaler', StandardScaler()),
    ('KNeighborsClassifier', KNeighborsClassifier()),
])
classify_and_plot(pipeline)

## Logistic Regression Classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
pipeline = Pipeline([
    ('select', feature_select),
    ('LogisticRegression', LogisticRegression(solver='liblinear')),
])
classify_and_plot(pipeline)

## Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB
pipeline = Pipeline([
    ('select', feature_select),
    ('GaussianNB_30', GaussianNB(priors=[0.75, 0.25])),
])
classify_and_plot(pipeline)

In [ ]:
from sklearn.naive_bayes import GaussianNB
pipeline = Pipeline([
    ('GaussianNB_80', GaussianNB(priors=[0.75, 0.25])),
])
classify_and_plot(pipeline)

## Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
pipeline = Pipeline([
    ('select', feature_select),
    ('DecisionTreeClassifier', DecisionTreeClassifier()),
])
classify_and_plot(pipeline)

If machine has Graphviz (https://www.graphviz.org/) installed, uncomment to render the decision tree as a PDF

In [ ]:
'''#!{sys.executable} -m pip install --quiet pydotplus
from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz
import pydotplus

dot_data = StringIO()
export_graphviz(pipeline.steps[-1][1], out_file=dot_data, filled=True, rounded=True, special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_pdf("models/tree.pdf")'''

# Ensemble Classifiers

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
pipeline = Pipeline([
    ('select', feature_select),
    ('RandomForestClassifier', RandomForestClassifier(n_estimators=20)),
])
classify_and_plot(pipeline)

## AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
pipeline = Pipeline([
    ('select', feature_select),
    ('AdaBoostClassifier', AdaBoostClassifier(n_estimators=100)),
])
classify_and_plot(pipeline)

## XGBoost

In [ ]:
from xgboost.sklearn import XGBClassifier
pipeline = Pipeline([
    ('select', feature_select),
    ('XGBClassifier', XGBClassifier(max_depth=100)),
])
classify_and_plot(pipeline)

## Microsoft LightGBM

In [ ]:
from lightgbm.sklearn import LGBMClassifier
pipeline = Pipeline([
    ('select', feature_select),
    ('LightGBM', LGBMClassifier()),
])
classify_and_plot(pipeline)

# Support Vector Machines

## Linear SVM

In [ ]:
from sklearn.svm import LinearSVC
pipeline = Pipeline([
    ('select', feature_select),
    ('scaler', StandardScaler()),
    ('LinearSVC', LinearSVC(C=0.2)),
])
classify_and_plot(pipeline)

## SVC with Polynomial Kernel

In [ ]:
from sklearn.svm import SVC
pipeline = Pipeline([
    ('select', feature_select),
    ('scaler', StandardScaler()),
    ('SVC_poly', SVC(C=1.0, kernel='poly', gamma='scale')),
])
classify_and_plot(pipeline)

## SVC with Radial Basis Function Kernel

In [ ]:
from sklearn.svm import SVC
pipeline = Pipeline([
    ('select', feature_select),
    ('scaler', StandardScaler()),
    ('SVC_rbf', SVC(C=1.0, kernel='rbf', gamma='scale')),
])
classify_and_plot(pipeline)

# Neural Networks

## Multi-Layer Perceptron

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras_metrics

Fully connected layers:
- 80 inputs
- Hidden layer with 40 neurons, ReLU as activation, and dropout rate of 10%
- Hidden layer with 20 neurons, ReLU as activation, and dropout rate of 10%
- 1 single output with sigmoid activation

In [ ]:
def create_model():
    model = Sequential()
    model.add(Dense(40, kernel_initializer='normal', activation='relu', input_dim=X_train.shape[1]))
    #model.add(LeakyReLU(alpha=0.001))
    model.add(Dropout(rate=0.1))
    model.add(Dense(20, kernel_initializer='normal', activation='relu'))
    #model.add(LeakyReLU(alpha=0.001))
    model.add(Dropout(rate=0.1))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[
        'accuracy', 
        #keras_metrics.precision(), 
        #keras_metrics.recall()
    ])
    return model

Features must be scaled by subtracting the mean and scaling to unit variance.

Absolute maximum of 250 training epochs, with 2 callbacks:
1. Monitor loss on validation test set (val_loss), and stop training once it stops improving
2. Persist to disk the model state that yielded the best val_loss.

In [ ]:
'''checkpoint = ModelCheckpoint('models/best_model.hdf5', monitor='val_acc',
                             mode='max', verbose=0, save_best_only=True)
early_stop = EarlyStopping(monitor='val_acc', patience=15, verbose=1, mode='max',
                          restore_best_weights=True)'''

checkpoint = ModelCheckpoint('models/best_model.hdf5', monitor='val_loss',
                             mode='min', verbose=0, save_best_only=True)
early_stop = EarlyStopping(monitor='val_loss', patience=50, verbose=1, mode='min',
                          restore_best_weights=True)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('KerasClassifier', KerasClassifier(build_fn=create_model, epochs=250, batch_size=5, verbose=1, 
                            callbacks=[checkpoint, early_stop], 
                            validation_data=(X_test, y_test==">50K"))),
])

In [ ]:
classify_and_plot(pipeline)